In this notebook, me and my team, we are going to scrape an ecommerce website. We'll get into each individual product page and retrieve our information from there. This is the website we are going to scrape <a href="https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt">Cdiscount</a>  – it's an French online shop.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Maintenant, nous allons définir l'URL de base de la page principale car nous en aurons besoin lorsque nous construirons nos URL pour chacun des produits individuels.

De plus, nous enverrons un agent utilisateur sur chaque requête HTTP, car si nous avons effectuez une requête GET à l'aide de requests , l'agent utilisateur est Python par défaut, ce qui peut être bloqué.

Donc, pour remplacer cela, nous allons déclarer une variable qui stockera notre user-agent.

In [2]:
baseurl = "https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [7]:
k = requests.get('https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt').text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"cPdtItem jsPdtItem"})
soup


<!DOCTYPE html>

<html>
<head>
<title>Baleen - Error Page</title>
<meta content="width=device-width" name="viewport"/>
<meta charset="utf-8"/>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@100;400&amp;display=swap" rel="stylesheet"/>
<link href="/.well-known/baleen/error-pages-in/assets/style.css" referrerpolicy="no-referrer" rel="stylesheet"/>
</head>
<body>
<div>
<h1 class="error-title">Error 403</h1>
<div class="error-message">
        You are not authorised to access this content
      </div>
<div class="error-illustration">
<!-- device_ok.svg -->
<svg version="1.1" viewbox="0 0 146 194" width="146px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<title>9B720660-BC66-468F-8F46-DA5D9A35113E</title>
<g fill="none" fill-rule="evenodd" id="ð«-Page-d'erreur" stroke="none" stroke-width="1">
<g id="Error-403" transform="translate(-257.000000, -403.000000)">
<g id="Group-4" transform="translate(257.000000, 403.631164)">
<g fill="#000000